# <center>Data Wrangling</center>

<img src="../image/wrangling_DALLE.jpeg" width=30% align="right" style="in-line">

>*80% of data wrangling is spent cleaning the data.*
>
>*The other 20% is spent complaining about cleaning the data.*
>
>— Source: [Kyle Bradbury & Nick Eubank](https://www.practicaldatascience.org/notebooks/class_3/week_3/20_cleaning_identifying.html)


## Learning goals

1. Be able to perform web scraping across multiple pages.
2. Be able to load, explore, and understand the structure of datasets using Python.
3. Become familiar with common data wrangling tasks, including handling missing values, merging datasets, filtering, and sorting data.

## Agenda

1. [Review: web scraping](#1)
2. [Data wrangling](#2)

<a name="1"></a>
## Agenda 1. Web scraping

&#x1F4DD; **<font color=dodgerblue>FROM LAST WEEK: </font>** Here is an example project. We would like to find out what the European Union has done recently (let's say since 2024) to advance sustainable mobility and transport. One possible data source is the news we just scraped, but we need more information other than the title of the news.

So now please write some code to collect **the date, the title, the short description, the news type, and the link to the full text** of all news in 2024. Save the data to a **csv** file.

Here are some tips:
1. How many pages do you need to scrape? Observe how the web addresses change between the first page and the second.
2. Remember we have talked about **avoid overloading servers** in ethics. Make sure to use `time.sleep()`.

If you would like to challenge yourself, see if you can scrape the full text (not the short description) of the news. Try with one or two pieces of news would be enough.

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import pandas as pd

### Step 1: Send an HTTP request

In [ ]:
url = "https://transport.ec.europa.eu/news-events/news_en?page=0"
page = requests.get(url)

In [ ]:
print(page)

### Step 2: Parse the HTML content

In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
#print(soup)
#print(soup.prettify())

### Step 3: Locate the HTML elements containing the desired data

Chrome: `View` -> `Developer` -> `Inspect Elements`

In [ ]:
news = soup.find_all("article", class_="ecl-content-item")

In [ ]:
print(news)

In [ ]:
len(news)

### Step 4: Extract data

In [ ]:
for item in news:
    date = item.time.attrs['datetime']
    title = item.find("a", class_="ecl-link ecl-link--standalone").get_text()
    desc = item.find("div", class_="ecl-content-block__description").get_text()
    news_type = item.find("li", class_="ecl-content-block__primary-meta-item").get_text()
    link = item.find("a", class_="ecl-link ecl-link--standalone", href=True)["href"]
    print(date,title,desc)
    print(news_type)
    print(link)
    print("=====")

In [ ]:
# news[0].time # the <time> element of the first article

### Save data to csv

To scrape multiple pages, we need to observe how the web addresses change.

In [ ]:
#page_number = range(12,22) # year 2024. your number could be different
page_number = range(0,13) # year 2025
stem = "https://transport.ec.europa.eu/news-events/news_en?page="

In [ ]:
#csv_filename = "news2024.csv" # give the csv file a name
csv_filename = "news2025.csv"

In [ ]:
for i in page_number: # loop over all the pages
    url = stem + str(i)
    print(url)
    
    page = requests.get(url) # send the request
    soup = BeautifulSoup(page.content, 'html.parser') # parse the content
    news = soup.find_all("article", class_="ecl-content-item") # locate all the news
    
    for item in news: # loop over each article
        try: # use try except to skip error e.g., NoneType object - empty elements
            date = item.time.attrs['datetime']
        except (AttributeError, KeyError, TypeError) as e:
            date = ""
            print(f"Error extracting date: {e}")
            print(item)
            print("====")
        try:
            title = item.find("a", class_="ecl-link ecl-link--standalone").get_text()
        except AttributeError as e:
            title = ""
            print(f"Error extracting title: {e}")
            print(item)
            print("====")
        try:
            desc = item.find("div", class_="ecl-content-block__description").get_text()
        except AttributeError as e:
            desc = ""
            print(f"Error extracting description: {e}")
            print(item)
            print("====")
        try:
            news_type = item.find("li", class_="ecl-content-block__primary-meta-item").get_text()
        except AttributeError as e:
            news_type = ""
            print(f"Error extracting news type: {e}")
            print(item)
            print("====")
        try:
            link = item.find("a", class_="ecl-link ecl-link--standalone", href=True)["href"]
        except (AttributeError, TypeError) as e:
            link = ""
            print(f"Error extracting link: {e}")
            print(item)
            print("====")
        csv.writer(open(csv_filename, "a", encoding="utf-8")).writerow([date, title, desc, news_type, link])
        
    time.sleep(2)

I got 2 errors running the above cell (for year 2025). It is okay. I have saved the errors.

In [ ]:
# the error output

<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2025-08-05T12:00:00Z">5 August 2025</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://aa293.referrals.selectminds.com/eu-rail/jobs/communication-and-ex-post-audit-officer-148"><span class="ecl-link__label">Vacancy notice: communication and ex-post audit officer</span><span class="wt-icon--external ecl-icon ecl-icon--2xs ecl-link__icon ecl-icon--external"></span></a></div><div class="ecl-content-block__description"><p>The Europe's Rail Joint Undertaking (EU-RAIL), based in Brussels is establishing a reserve list for a post of communication and ex-post audit officer (temporary agent, grade AD5)</p></div><ul class="ecl-content-block__secondary-meta-container"><li class="ecl-content-block__secondary-meta-item"><span class="wt-icon--clock ecl-icon ecl-icon--s ecl-content-block__secondary-meta-icon ecl-icon--clock"></span><span class="ecl-content-block__secondary-meta-label">1 min read</span></li></ul></div></article>
====

<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2025-08-05T12:00:00Z">5 August 2025</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://aa293.referrals.selectminds.com/eu-rail/jobs/communication-and-ex-post-audit-officer-148"><span class="ecl-link__label">Vacancy notice: communication and ex-post audit officer</span><span class="wt-icon--external ecl-icon ecl-icon--2xs ecl-link__icon ecl-icon--external"></span></a></div><div class="ecl-content-block__description"><p>The Europe's Rail Joint Undertaking (EU-RAIL), based in Brussels is establishing a reserve list for a post of communication and ex-post audit officer (temporary agent, grade AD5)</p></div><ul class="ecl-content-block__secondary-meta-container"><li class="ecl-content-block__secondary-meta-item"><span class="wt-icon--clock ecl-icon ecl-icon--s ecl-content-block__secondary-meta-icon ecl-icon--clock"></span><span class="ecl-content-block__secondary-meta-label">1 min read</span></li></ul></div></article>


<a name="2"></a>
## Agenda 2. Data wrangling

Data wrangling is the process of converting raw data into a usable form. It typically involves examining the data, handling missing values, cleaning, and transforming it.

### Examining data

We start by loading our datasets and inspecting them to get a better sense of their structure. 

In [ ]:
news2025 = pd.read_csv("news2025.csv", encoding="utf-8", names=["date", "title", "desc", "type", "link"]) # import csv with column names
news2024 = pd.read_csv("news2024.csv", encoding="utf-8", names=["date", "title", "desc", "type", "link"])

In [ ]:
news2024.head()

In [ ]:
len(news2024) # check the number of rows

In [ ]:
news2024.shape # check the number of rows and columns

In [ ]:
news2024.info() # summarize by the columns

In [ ]:
# news2024.describe() # useful for numerical data

In [ ]:
type(news2024.date.iloc[0]) # check the data type

In [ ]:
news2024.type.unique() # check the unique values of a column

In [ ]:
news2024.type.value_counts() # check each value's number of occurrences

### Handling missing values

In [ ]:
news2024.isnull().sum() # identify missing data

In [ ]:
news2025.isnull().sum()

In [ ]:
news2025[news2025.title.isnull()] # pinpoint which rows have missing values

In [ ]:
#news2025.title.isnull() # this is a mask

In [ ]:
news2025[news2025.link.isnull()] # check again by "link"

Missing values can often skew analyses or create errors during analysis. Here are some common strategies for handling missing values:

- Manually review and fix the missing data (if possible)
- Remove missing data e.g., drop rows
- Impute missing data e.g., replace with the mean (for numerical data)
- Fill with a specific value e.g., replace with "unknown" or "NA"
- Predictive imputation e.g., use machine learning models to predict missing values based on other features
- Leave as missing

For our example, we choose the first option because we have access to the original data source. Let's go back to the previous error messages.

&#x1F4A1; **Markdown**: Markdown is a markup language. It can be used as a text-to-HTML conversion tool. Read more on Jupyter [Markdown cells](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html). It can help us read the error messages and fix the missing values. Below is a direct copy & paste of the [error messages](#error) (in a Markedown cell).

#### the error output

<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2025-08-05T12:00:00Z">5 August 2025</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://aa293.referrals.selectminds.com/eu-rail/jobs/communication-and-ex-post-audit-officer-148"><span class="ecl-link__label">Vacancy notice: communication and ex-post audit officer</span><span class="wt-icon--external ecl-icon ecl-icon--2xs ecl-link__icon ecl-icon--external"></span></a></div><div class="ecl-content-block__description"><p>The Europe's Rail Joint Undertaking (EU-RAIL), based in Brussels is establishing a reserve list for a post of communication and ex-post audit officer (temporary agent, grade AD5)</p></div><ul class="ecl-content-block__secondary-meta-container"><li class="ecl-content-block__secondary-meta-item"><span class="wt-icon--clock ecl-icon ecl-icon--s ecl-content-block__secondary-meta-icon ecl-icon--clock"></span><span class="ecl-content-block__secondary-meta-label">1 min read</span></li></ul></div></article>
====

<article class="ecl-content-item"><div class="ecl-content-block ecl-content-item__content-block" data-ecl-auto-init="ContentBlock" data-ecl-content-block=""><ul class="ecl-content-block__primary-meta-container"><li class="ecl-content-block__primary-meta-item">News article</li><li class="ecl-content-block__primary-meta-item"><time datetime="2025-08-05T12:00:00Z">5 August 2025</time></li></ul><div class="ecl-content-block__title" data-ecl-title-link=""><a class="ecl-link ecl-link--standalone ecl-link--icon" href="https://aa293.referrals.selectminds.com/eu-rail/jobs/communication-and-ex-post-audit-officer-148"><span class="ecl-link__label">Vacancy notice: communication and ex-post audit officer</span><span class="wt-icon--external ecl-icon ecl-icon--2xs ecl-link__icon ecl-icon--external"></span></a></div><div class="ecl-content-block__description"><p>The Europe's Rail Joint Undertaking (EU-RAIL), based in Brussels is establishing a reserve list for a post of communication and ex-post audit officer (temporary agent, grade AD5)</p></div><ul class="ecl-content-block__secondary-meta-container"><li class="ecl-content-block__secondary-meta-item"><span class="wt-icon--clock ecl-icon ecl-icon--s ecl-content-block__secondary-meta-icon ecl-icon--clock"></span><span class="ecl-content-block__secondary-meta-label">1 min read</span></li></ul></div></article>


In [ ]:
news2025.title.iloc[36]# the first missing title

In [ ]:
# write the title to the cell
news2025.iloc[36].title = "Vacancy notice: communication and ex-post audit officer"

In [ ]:
news2025[news2025.title.isnull()] # check missing titles again

In [ ]:
# do the same for the missing link
news2025.iloc[36].link = "https://aa293.referrals.selectminds.com/eu-rail/jobs/communication-and-ex-post-audit-officer-148"

In [ ]:
news2025.isnull().sum() # check again. no more missing values

### Merging datasets

There are many different ways of merging datasets. Check this [documentation](https://pandas.pydata.org/docs/user_guide/merging.html). Let's combine our two datasets into one.

In [ ]:
news = pd.concat([news2024, news2025], ignore_index=True) # concatenate and re-index

In [ ]:
news

In [ ]:
news.isnull().sum()

Nothing is missing. However, we might have created duplicated values when merging datasets.

### Cleaning

In [ ]:
news[news.duplicated()] # identify duplicated rows

In [ ]:
news.duplicated() # returns a Boolean Series indicating whether each row is a duplicate of a previous row

In [ ]:
news = news.drop_duplicates() # drop duplicated rows

In [ ]:
news

We also want to fix the date using [`pandas.to_datetime` function](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html). 

In [ ]:
type(news.date[0])

In [ ]:
news.loc[:, "date"] = pd.to_datetime(news.date) # change the data type to date

In [ ]:
type(news.date[0])

In [ ]:
news

In [ ]:
news = news.sort_values(by="date", ascending=False) # sort the rows by date

In [ ]:
news

In [ ]:
news['date'] = pd.to_datetime(news['date']) # make sure the 'date' column is of datetime type
news = news[news['date'].dt.year!=2023] # remove news from 2023

In [ ]:
news

In [ ]:
news.type.unique()

In [ ]:
news[news.type=="Press release"]

In [ ]:
#news[news.type=="General publications"]

Let's only keep the news.

In [ ]:
news = news[news.type=="News article"]

In [ ]:
news = news.reset_index(drop=True) # reset the index

In [ ]:
news

In [ ]:
#news = news[["date", "title", "desc"]] # new df with selected columns (if needed)

In [ ]:
news.to_csv("news_clean.csv", index=False) # save to csv

&#x2753;**<font color=cornflowerblue>QUESTION: </font>** What are the common tasks involved in data wrangling?

&#x270A; **<font color=firebrick>DO THIS: </font>** Here are two options to practice what we've learned and to further develop your data skills:

1. Do some data wrangling on the data you scraped last week.
2. Explore the existing `news_clean.csv` dataset. What research question can it answer? Using what methods/tools?

Feel free to pick one task or do both. Have fun!

In [ ]:
# put your code here













---------
### Congratulations, we are done!

This notebook is written by [Meng Cai](https://www.verkehr.tu-darmstadt.de/vv/das_institut_ivv/team_ivv/wissenschaftliche_mitarbeiter_doktoranden/meng_cai/standardseite_204.de.jsp), Technical University of Darmstadt. This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a>